### iterated greedy method for the distributed permutation flowshop scheduling problem can be modified to handle multiple factories

In [5]:
import numpy as np

# define the number of machines and jobs
num_factories = 2
num_machines = 3
num_jobs = 5

# define the processing time for each job on each machine at each factory
processing_times = np.random.randint(low=1, high=10, size=(num_factories, num_machines, num_jobs))
print(processing_times)
# define the initial permutation of jobs
permutation = np.random.permutation(num_jobs)

# iterated greedy method to find the optimal permutation
best_makespan = np.inf
best_permutation = permutation
for i in range(num_jobs):
    for j in range(i+1, num_jobs):
        # swap jobs i and j in the permutation
        permutation[[i,j]] = permutation[[j,i]]

        # calculate the makespan for the current permutation
        makespan = np.zeros((num_factories, num_machines))
        for f in range(num_factories):
            for k in range(num_jobs):
                job = permutation[k]
                makespan[f, 0] += processing_times[f, 0, job]
                for m in range(1, num_machines):
                    makespan[f, m] = max(makespan[f, m], makespan[f, m-1]) + processing_times[f, m, job]
        total_makespan = max(makespan[:,-1])
        # check if the current permutation is better than the previous best
        if total_makespan < best_makespan:
            best_permutation = permutation
            best_makespan = total_makespan

        # swap jobs i and j back in the permutation
        permutation[[i,j]] = permutation[[j,i]]

# print the best permutation and makespan
print("Best permutation:", best_permutation)
print("Best makespan:", best_makespan)

[[[1 3 6 2 5]
  [1 8 2 7 4]
  [8 5 8 7 7]]

 [[7 6 6 2 2]
  [7 4 5 1 7]
  [2 8 9 8 9]]]
Best permutation: [0 3 4 2 1]
Best makespan: 45.0


### Scheduling the distributed assembly flowshop problem to minimize the makespan using genetic algorithms

## bounded-search iterated greedy algorithm BSIG and four local search methods are designed to improve solution quality.

In [11]:
import numpy as np
from deap import base, creator, tools,algorithms

# define the number of factories, machines, and jobs
num_factories = 2
num_machines = 3
num_jobs = 5

# define the processing time for each job on each machine at each factory
processing_times = np.random.randint(low=1, high=10, size=(num_factories, num_machines, num_jobs))

# define the fitness function
def fitness(individual):
    makespan = np.zeros((num_factories, num_machines))
    for f in range(num_factories):
        for i in range(num_jobs):
            job = individual[i]
            makespan[f, 0] += processing_times[f, 0, job]
            for m in range(1, num_machines):
                makespan[f, m] = max(makespan[f, m], makespan[f, m-1]) + processing_times[f, m, job]
    return max(makespan[:,-1]),

# define the genetic algorithm
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("permutation", np.random.permutation, num_jobs)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.permutation)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

pop = toolbox.population(n=50)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, stats=stats, halloffame=hof)

# print the best solution
print("Best individual:", hof[0])
print("Makespan:", fitness(hof[0])[0])

gen	nevals	avg  	std   	min	max
0  	50    	42.46	1.9719	39 	46 
1  	32    	40.58	1.84488	39 	45 
2  	30    	39.64	1.35292	39 	44 
3  	30    	39.58	1.66241	39 	46 
4  	31    	39.42	1.25044	39 	45 
5  	27    	39.34	1.29012	39 	45 
6  	30    	39.32	1.20731	39 	45 
7  	27    	39.56	1.71067	39 	46 
8  	32    	39.06	0.42   	39 	42 
9  	26    	39.24	0.884534	39 	43 
10 	25    	39.36	1.22898 	39 	44 
11 	33    	39.12	0.84    	39 	45 
12 	26    	39.04	0.28    	39 	41 
13 	29    	39.14	0.693109	39 	43 
14 	30    	39.08	0.44    	39 	42 
15 	25    	39   	0       	39 	39 
16 	34    	39.12	0.620967	39 	43 
17 	24    	39   	0       	39 	39 
18 	27    	39.08	0.56    	39 	43 
19 	24    	39.12	0.84    	39 	45 
20 	33    	39.2 	0.8     	39 	43 
21 	32    	39   	0       	39 	39 
22 	27    	39   	0       	39 	39 
23 	23    	39   	0       	39 	39 
24 	22    	39.22	0.878408	39 	43 
25 	32    	39   	0       	39 	39 
26 	30    	39.32	1.0284  	39 	43 
27 	28    	39.02	0.14    	39 	40 
28 	29    	39.08	0.56    	